In [1]:
#Import necessary libaries
import pandas as pd
import numpy as np

#Load the dataset
cm = pd.read_csv('Churn_Modeling.csv')

cm

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

# Load the dataset
cm = pd.read_csv('Churn_Modeling.csv')

# Drop unnecessary columns (RowNumber, CustomerId, Surname)
cm = cm.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

# Separate features (X) and target variable (y)
X = cm.drop('Exited', axis=1)
y = cm['Exited']

# Define categorical and numerical features
categorical_features = ['Geography', 'Gender', 'HasCrCard', 'IsActiveMember']
numerical_features = ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'EstimatedSalary']

# Create transformers for preprocessing
# One-hot encode categorical variables and normalize numerical variables
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(), categorical_features)
    ])

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a pipeline for preprocessing
pipeline = Pipeline(steps=[('preprocessor', preprocessor)])

# Fit and transform the training data
X_train_preprocessed = pipeline.fit_transform(X_train)

# Transform the test data using the same preprocessing steps
X_test_preprocessed = pipeline.transform(X_test)

# Get the names of the numerical features after preprocessing
num_feature_names = pipeline.named_steps['preprocessor'].named_transformers_['num'].get_feature_names_out(input_features=numerical_features)

# Get the names of the categorical features after preprocessing
cat_feature_names = pipeline.named_steps['preprocessor'].named_transformers_['cat'].get_feature_names_out(input_features=categorical_features)

# Concatenate the numerical and categorical feature names
preprocessed_columns = list(num_feature_names) + list(cat_feature_names)

# Create DataFrames for the transformed data
X_train_preprocessed_cm = pd.DataFrame(X_train_preprocessed, columns=preprocessed_columns)
X_test_preprocessed_cm = pd.DataFrame(X_test_preprocessed, columns=preprocessed_columns)

# Display the first few rows of the transformed data
print("Transformed Training Data:")
print(X_train_preprocessed_cm.head())

print("\nTransformed Test Data:")
print(X_test_preprocessed_cm.head())


Transformed Training Data:
   CreditScore       Age    Tenure   Balance  NumOfProducts  EstimatedSalary  \
0     0.356500 -0.655786  0.345680 -1.218471       0.808436         1.367670   
1    -0.203898  0.294938 -0.348369  0.696838       0.808436         1.661254   
2    -0.961472 -1.416365 -0.695393  0.618629      -0.916688        -0.252807   
3    -0.940717 -1.131148  1.386753  0.953212      -0.916688         0.915393   
4    -1.397337  1.625953  1.386753  1.057449      -0.916688        -1.059600   

   Geography_France  Geography_Germany  Geography_Spain  Gender_Female  \
0               1.0                0.0              0.0            0.0   
1               0.0                1.0              0.0            0.0   
2               0.0                0.0              1.0            0.0   
3               1.0                0.0              0.0            1.0   
4               1.0                0.0              0.0            0.0   

   Gender_Male  HasCrCard_0  HasCrCard_1  IsAct

In [3]:
!pip install -U scikit-learn imbalanced-learn
!pip install imbalanced-learn
!pip install --upgrade scikit-learn
!pip install --upgrade imbalanced-learn


In [4]:
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

# Assuming you have already loaded and preprocessed your data
# (X_train_preprocessed, X_test_preprocessed, y_train, y_test)

# Use SMOTE to oversample the minority class in the training set
smote = SMOTE(random_state=42)
X_train_preprocessed_smote, y_train_smote = smote.fit_resample(X_train_preprocessed, y_train)

# Create an SVM model
svm_model = SVC(kernel='linear', random_state=42)

# Train the model on the oversampled training data
svm_model.fit(X_train_preprocessed_smote, y_train_smote)

# Make predictions on the preprocessed test data
y_pred_test = svm_model.predict(X_test_preprocessed)
y_pred_train = svm_model.predict(X_train_preprocessed_smote)  # Use the oversampled training data for predictions

# Evaluate the model
accuracy_test = accuracy_score(y_test, y_pred_test)
conf_matrix_test = confusion_matrix(y_test, y_pred_test)
classification_rep_test = classification_report(y_test, y_pred_test)

# Display evaluation metrics
print(f"Accuracy (Test): {accuracy_test:.4f}")
print("\nConfusion Matrix (Test):")
print(conf_matrix_test)
print("\nClassification Report (Test):\n")
print(classification_rep_test)



Accuracy (Test): 0.7275

Confusion Matrix (Test):
[[1168  439]
 [ 106  287]]

Classification Report (Test):

              precision    recall  f1-score   support

           0       0.92      0.73      0.81      1607
           1       0.40      0.73      0.51       393

    accuracy                           0.73      2000
   macro avg       0.66      0.73      0.66      2000
weighted avg       0.81      0.73      0.75      2000



In [6]:
from sklearn.metrics import f1_score, classification_report

# Compute F1-scores for the linear kernel model
f1_test_linear = f1_score(y_test, y_pred_test, average=None)
f1_train_linear = f1_score(y_train, y_pred_train, average=None)

# Generate a classification report for test data
classification_rep_linear = classification_report(y_test, y_pred_test)

# Print F1-scores and the classification report
print("F1-scores (Linear Kernel):", f1_test_linear)
print("\nClassification Report (Linear Kernel):\n", classification_rep_linear)


F1-scores (Linear Kernel): [0.89104519 0.        ]

Classification Report (Linear Kernel):
               precision    recall  f1-score   support

           0       0.80      1.00      0.89      1607
           1       0.00      0.00      0.00       393

    accuracy                           0.80      2000
   macro avg       0.40      0.50      0.45      2000
weighted avg       0.65      0.80      0.72      2000



C:\Users\kxion\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\kxion\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\kxion\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [5]:
# Testing a range of various C-Values & Comparing Performance Metrics at those corresponding values

import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score

# Define the range of C values to be tested
C_values = [0.1, 0.5, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 50, 100, 250]

# Initialize lists to store results
train_accuracies = []
test_accuracies = []
f1_scores_class1 = []

# Loop over each value of C to train the SVM model and evaluate performance
for C in C_values:
    # Create and train the SVM model
    svm_model_loop = SVC(C=C, kernel='linear', random_state=42)
    svm_model_loop.fit(X_train_preprocessed, y_train)

    # Predictions and evaluation on training set
    y_pred_train_loop = svm_model_loop.predict(X_train_preprocessed)
    train_accuracies.append(accuracy_score(y_train, y_pred_train_loop))

    # Predictions and evaluation on test set
    y_pred_test_loop = svm_model_loop.predict(X_test_preprocessed)
    test_accuracies.append(accuracy_score(y_test, y_pred_test_loop))
    f1_scores_class1.append(f1_score(y_test, y_pred_test_loop, pos_label=1))

# Compile results into a DataFrame for better visualization
results_cm = pd.DataFrame({
    'C_value': C_values,
    'Train_Accuracy': train_accuracies,
    'Test_Accuracy': test_accuracies,
    'F1_Score_Class1': f1_scores_class1
})

# Display the results
print(results_cm)


    C_value  Train_Accuracy  Test_Accuracy  F1_Score_Class1
0       0.1          0.7945         0.8035              0.0
1       0.5          0.7945         0.8035              0.0
2       1.0          0.7945         0.8035              0.0
3       2.0          0.7945         0.8035              0.0
4       3.0          0.7945         0.8035              0.0
5       4.0          0.7945         0.8035              0.0
6       5.0          0.7945         0.8035              0.0
7       6.0          0.7945         0.8035              0.0
8       7.0          0.7945         0.8035              0.0
9       8.0          0.7945         0.8035              0.0
10      9.0          0.7945         0.8035              0.0
11     10.0          0.7945         0.8035              0.0
12     50.0          0.7945         0.8035              0.0
13    100.0          0.7945         0.8035              0.0
14    250.0          0.7945         0.8035              0.0


In [4]:
# Hyper Parameter Tuning

from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform

# Example: Using RandomizedSearchCV for hyperparameter tuning
param_dist = {
    'C': uniform(loc=0.1, scale=20),
    'kernel': ['linear', 'rbf'],
    'gamma': ['scale', 'auto'] + list(uniform(loc=0.1, scale=1).rvs(20)),
    'degree': [2, 3, 4]
}

svm_model = SVC(random_state=42)
random_search = RandomizedSearchCV(svm_model, param_distributions=param_dist, n_iter=20, cv=3, random_state=42)
random_search.fit(X_train_preprocessed, y_train)

# Best hyperparameters
best_params = random_search.best_params_
print("Best Hyperparameters:", best_params)


Best Hyperparameters: {'C': 3.7364993441420125, 'degree': 2, 'gamma': 'scale', 'kernel': 'rbf'}


In [7]:
# SVM Model for best Hyperparamteres

import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

# Assuming you have already loaded and preprocessed your data
# (X_train_preprocessed, X_test_preprocessed, y_train, y_test)

# Define the best hyperparameters
best_hyperparameters = {'C': 3.7364993441420125, 'degree': 2, 'gamma': 'scale', 'kernel': 'rbf'}

# Create a pipeline for preprocessing with the best hyperparameters
best_preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(), categorical_features)
    ])

# Create an SVM model with the best hyperparameters
best_svm_model = SVC(**best_hyperparameters, random_state=42)

# Create a pipeline for the complete model with preprocessing and SVM
best_model_pipeline = Pipeline(steps=[
    ('preprocessor', best_preprocessor),
    ('svm_model', best_svm_model)
])

# Fit the model on the training data
best_model_pipeline.fit(X_train, y_train)

# Make predictions on the test set
y_pred_test_best = best_model_pipeline.predict(X_test)

# Evaluate the model
accuracy_best = accuracy_score(y_test, y_pred_test_best)
conf_matrix_best = confusion_matrix(y_test, y_pred_test_best)
classification_rep_best = classification_report(y_test, y_pred_test_best)

# Display the results
print(f"Best Model Hyperparameters: {best_hyperparameters}")
print(f"\nAccuracy (Best Model): {accuracy_best:.4f}")
print("\nConfusion Matrix (Best Model):")
print(conf_matrix_best)
print("\nClassification Report (Best Model):\n", classification_rep_best)


Best Model Hyperparameters: {'C': 3.7364993441420125, 'degree': 2, 'gamma': 'scale', 'kernel': 'rbf'}

Accuracy (Best Model): 0.8670

Confusion Matrix (Best Model):
[[1555   52]
 [ 214  179]]

Classification Report (Best Model):
               precision    recall  f1-score   support

           0       0.88      0.97      0.92      1607
           1       0.77      0.46      0.57       393

    accuracy                           0.87      2000
   macro avg       0.83      0.71      0.75      2000
weighted avg       0.86      0.87      0.85      2000

